In [2]:
import pandas as pd
import sklearn as sk
import sklearn.model_selection

In [7]:
# load data
data = pd.read_csv('../unmodified_raw/AI_Human.csv')

data

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0
...,...,...
487230,Tie Face on Mars is really just a big misunder...,0.0
487231,The whole purpose of democracy is to create a ...,0.0
487232,I firmly believe that governments worldwide sh...,1.0
487233,I DFN't agree with this decision because a LFT...,0.0


In [8]:
# X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(data['text'], data['generated'], test_size=0.2)

import tensorflow as tf

In [15]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

X, Y = data['text'], data["generated"]

# Convert labels to numeric values
unique_labels = Y.unique()
label_map = {label: i for i, label in enumerate(unique_labels)}
Y = Y.map(label_map)

# Tokenize the text data
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)


# Pad the sequences to have the same length
X_seq = tf.keras.preprocessing.sequence.pad_sequences(X_seq)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_seq, Y, test_size=0.2)

: 

In [ ]:

print(len(X_train), len(X_test), len(Y_train), len(Y_test))
# Tokenize the words
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding sequences to ensure uniform input size
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, padding='post')
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, padding='post')

In [ ]:
# # Define a simple RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=16),
    tf.keras.layers.SimpleRNN(units=128),
    tf.keras.layers.Dense(2, activation='softmax')
])

# load model
#model = tf.keras.models.load_model('20240313-151915_rnn.h5')

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(X_train_padded.shape, Y_train.shape)
print(X_train_padded)

# Train the model
model.fit(X_train_padded, Y_train, epochs=150, validation_data=(X_test_padded, Y_test))

# Save model with current date and parameters as filename
import datetime
filename = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '_rnn.h5'
model.save(filename)
